The the [day 9 puzzle](https://adventofcode.com/2021/day/9) involves analyzing a grid to discover "low points" in the grid and then extrapolate out "basins" from those low points.

A low point is when no adjacent numbers are lower than it.  A basin is all points touching a low point (or parts of the basin) that increment up in value without hitting 9.

In [1]:
import gridthings
import pandas  # only for printing out the grid, not for analysis

In [2]:
data = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""

# Use an IntGrid vice regular Grid to cast each value to int
#
# Setting the out_of_bounds_value to 10 so that when we
# compare whether one cell is less than a neighbor cell,
# any out of bounds cells will return true.
# you'll see how this applies down below
grid = gridthings.IntGrid(data, out_of_bounds_value=10)
grid

<IntGrid shape=(5, 10)>

In [3]:
import pandas

df = pandas.DataFrame(grid.data)
# each cell in the dataframe is a gridthings.Cell pydantic object
# turn that into just the Cell.value for easier display
df = df.applymap(lambda cell: cell.value)

In [4]:
# Set the grids active context to some cell (it'll return the Cell object)
grid.enter(y=1, x=2)

IntCell(y=1, x=2, value=8)

In [5]:
# Look at what .peek looks like.  .peek_linear returns left/right/up/down
grid.peek_linear()

[IntCell(y=1, x=1, value=9),
 IntCell(y=1, x=3, value=7),
 IntCell(y=0, x=2, value=9),
 IntCell(y=2, x=2, value=5)]

In [6]:
# also look at what a peek next to an out of bounds cell would be
grid.enter(0, 0)
grid.peek_linear()

[OutOfBoundsCell(y=0, x=-1, value=10),
 IntCell(y=0, x=1, value=1),
 OutOfBoundsCell(y=-1, x=0, value=10),
 IntCell(y=1, x=0, value=3)]

In [7]:
# observe that you can compare cells directly
cell1 = grid.enter(0, 0)
cell2 = grid.enter(0, 1)
cell1, cell2

(IntCell(y=0, x=0, value=2), IntCell(y=0, x=1, value=1))

In [8]:
cell1 == cell2

False

In [9]:
cell1 > cell2

True

In [10]:
cell2 < cell1

True

In [11]:
# Lastly, check out what a flattened dictionary looks like
grid.flatten()[:5]

[IntCell(y=0, x=0, value=2),
 IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=2, value=9),
 IntCell(y=0, x=3, value=9),
 IntCell(y=0, x=4, value=9)]

In [12]:
# Step one, find all the low points in the grid
low_points = []


def is_low_point(y, x):
    seed = grid.enter(y, x)
    for cell in grid.peek_linear():
        # return False if any neighboring cells are smaller
        # it wouldn't be a low point if that's the case
        # remember OutOfBoundCells have value 10 because
        # of the default set earlier on
        if seed > cell:
            return False
    return True


for cell in grid.flatten():
    if is_low_point(cell.y, cell.x):
        low_points.append(cell)

low_points

[IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=9, value=0),
 IntCell(y=2, x=2, value=5),
 IntCell(y=4, x=6, value=5)]

In [13]:
# Step two, find basins around those low points
# a basin is all locations neighboring a low point
# that are higher value but not OutOfBounds or value 9
#
# Note that Cells can be compared with >, >=, <, <=
# as if they were plain values, but Cell == Cell
# will compare the dictionaries, so "cell in [cells]"
# still works here

basins = []


class Basin:
    def __init__(self, start_cell: gridthings.Cell):
        self.points = [start_cell]
        for point in self.points:
            # self.points will be added to if appropriate
            # in the self.explore() method
            self.explore(point)

    def explore(self, seed_cell: gridthings.Cell):
        # set active context for .peek() and return
        # Cell at this y, x point
        seed = grid.enter(seed_cell.y, seed_cell.x)
        for cell in grid.peek_linear():
            if cell not in self.points:
                if cell < 9 and cell > seed:
                    self.points.append(cell)

    def __repr__(self):
        return f"<Basin start={self.points[0]} points={len(self.points)}>"


for point in low_points:
    basin = Basin(start_cell=point)
    basins.append(basin)

basins

[<Basin start=y=0 x=1 value=1 points=3>,
 <Basin start=y=0 x=9 value=0 points=9>,
 <Basin start=y=2 x=2 value=5 points=14>,
 <Basin start=y=4 x=6 value=5 points=9>]

In [14]:
basins[0].points

[IntCell(y=0, x=1, value=1),
 IntCell(y=0, x=0, value=2),
 IntCell(y=1, x=0, value=3)]

In [15]:
basins[1].points

[IntCell(y=0, x=9, value=0),
 IntCell(y=0, x=8, value=1),
 IntCell(y=1, x=9, value=1),
 IntCell(y=0, x=7, value=2),
 IntCell(y=1, x=8, value=2),
 IntCell(y=2, x=9, value=2),
 IntCell(y=0, x=6, value=3),
 IntCell(y=0, x=5, value=4),
 IntCell(y=1, x=6, value=4)]

In [16]:
# Find the three largest Basins and multiply their size together
sorted_basins = list(sorted(basins, key=lambda item: len(item.points)))
sorted_basins

[<Basin start=y=0 x=1 value=1 points=3>,
 <Basin start=y=0 x=9 value=0 points=9>,
 <Basin start=y=4 x=6 value=5 points=9>,
 <Basin start=y=2 x=2 value=5 points=14>]

In [17]:
import math

math.prod([len(basin.points) for basin in sorted_basins[-3:]])

1134